In [150]:
#IPython interpreter will import matplotlib and NumPy modules, convenient access to their functions
%pylab inline 
#for regex
import re
import math
import string
#collections : high performance container datatypes
from collections import Counter
from __future__ import division

#First we need some text,from a file. Then we can break the text into words
text_file = 'C:/Users/Hitarthi/Desktop/SEM 2/IDS/Homework 3/big.txt'
data = open(text_file).read()
len(data)

Populating the interactive namespace from numpy and matplotlib


C:\Users\Hitarthi\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['sample', 'test', 'product']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


6488665

In [151]:
#make tokens from the data
#ignore all the punctuation and numbers, and anything that is not a letter
#Normalize to lowercase


#re.findall(pattern, string, flags=0)
# '+' Causes the resulting RE to match 1 or more repetitions of the preceding RE.
# ab+ will match ‘a’ followed by any non-zero number of ‘b’s; it will not match just ‘a’.

def tokens(text):
    return re.findall('[a-z]+', text.lower()) 

In [152]:
tokens('This is: A test, 1, 2, 3, this is.')

['this', 'is', 'a', 'test', 'this', 'is']

In [153]:
WORDS = tokens(data)
len(WORDS)

1105285

In [154]:
#the first 10:
print(WORDS[:10])

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'adventures', 'of', 'sherlock', 'holmes']


In [155]:
#bag of words model, we ignore the order of words, but maintain their frequency
#function to sample an n word sentence from a bag of words(WORDS)

def sample(bag, n=10):
    return ' '.join(random.choice(bag) for _ in range(n))

sample(WORDS)

'which our clear kissed at pulled of the by patient'

In [156]:
#Another representation for a bag of words :
#a Counter, which is a dictionary of {'word': count} pairs. 

Counter(tokens('Is this a test? It is a test!'))

Counter({'a': 2, 'is': 2, 'it': 1, 'test': 2, 'this': 1})

In [157]:
#make a Counter for the big list of WORDS

COUNTS = Counter(WORDS)
print(COUNTS.most_common(10))

[('the', 80030), ('of', 40025), ('and', 38313), ('to', 28766), ('in', 22050), ('a', 21155), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]


In [158]:
for w in tokens('the rare and neverbeforeseen words'):
    print(w,COUNTS[w])

the 80030
rare 83
and 38313
neverbeforeseen 0
words 460


TASK : Given a word w, find the most likely correction c = correct(w).

Approach: Try all candidate words c that are known words that are near w. Choose the most likely one.

For now, in a trivial way: always prefer nearer, but when there is a tie on nearness, use the word with the highest WORDS count.

Measure nearness by edit distance: the minimum number of deletions, transpositions, insertions, or replacements of characters. 

By trial and error, we determine that going out to edit distance 2 will give us reasonable results. Then we can define correct(w):

In [159]:
def correct(word):
    #Generating all the words with edit distance of 0, 1 & 2
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=COUNTS.get)

In [160]:
def known(words):
    #Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    return {word}

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [161]:
def edits1(word):
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [162]:
splits('wird')

[('', 'wird'), ('w', 'ird'), ('wi', 'rd'), ('wir', 'd'), ('wird', '')]

In [163]:
print(edits0('wird'))

{'wird'}


In [164]:
print(len(edits1('wird')))

234


In [165]:
print(len(edits2('wird')))

24254


In [166]:
#re.sub(pattern, repl, string, count=0, flags=0) 
#Return the string obtained by replacing the leftmost non-overlapping occurrences of pattern in string by the replacement repl.
#If the pattern isn’t found, string is returned unchanged.

def correct_text(text):
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    word = match.group()
    return case_of(word)(correct(word.lower()))


#str.Title()
#parameters:str is a valid string which we need to convert.
#return: This function returns a string which has first letter in each word uppercase and remaining lowercase. 

def case_of(text):
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [167]:
correct_text('Speling Errurs IN somethink. Whutever; unusuel misteakes, Hitarthi ?')

'Spelling Errors IN something. Whatever; unusual mistakes, Hitarthi ?'

In the text "three, too, one, blastoff!" we might want to correct "too" with "two", even though "too" is in the dictionary. We can do better if we look at a sequence of words, not just an individual word one at a time. 


Counts to Probabilities of Word Sequences : 

In [168]:
# probability of a word, P(w).
# pdist:takes as input a Counter (a bag of words)
# returns a function that acts as a probability distribution over all possible words. 


def pdist(counter):
    "Make a probability distribution, given evidence from a Counter."
    N = sum(list(counter.values()))
    return lambda x: counter[x]/N

P = pdist(COUNTS)

In [169]:
for w in tokens('"The" is most common word in English'):
    print(P(w), w)

0.0724066643445 the
0.00884296810325 is
0.000821507574969 most
0.00025966153526 common
0.000269613719538 word
0.0199496057578 in
0.000190900989338 english


 what is the probability of a sequence of words? Use the definition of a joint probability:

P(w1…wn)=P(w1)×P(w2∣w1)×P(w3∣w1w2)…×…P(wn∣w1…wn−1)P(w1…wn)=P(w1)×P(w2∣w1)×P(w3∣w1w2)…×…P(wn∣w1…wn−1)

The bag of words model assumes that each word is drawn from the bag independently of the others. This gives us the wrong approximation:

P(w1…wn)=P(w1)×P(w2)×P(w3)…×…P(wn)P(w1…wn)=P(w1)×P(w2)×P(w3)…×…P(wn)

In [170]:
# "Probability of words, assuming each word is independent of others."

def Pwords(words):
    return product(P(w) for w in words)

# "Multiply the numbers together.
def product(nums):
    result = 1
    for x in nums:
        result *= x
    return result

In [171]:
tests = ['this is a test',
         'this is a unusual test','this is a neverbeforeseen test']

for test in tests:
    print(Pwords(tokens(test)), test)

2.9833963328e-11 this is a test
8.63747202302e-16 this is a unusual test
0.0 this is a neverbeforeseen test
